<a href="https://colab.research.google.com/github/Dhaksa/Vocal-Extraction/blob/main/Dataset_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import random
import zipfile
import shutil


In [ ]:
# Mount Google Drive (only needed if running in Google Colab)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Path to save directly in Google Drive
drive_path = "/content/drive/MyDrive/New/Clean-vocals"
os.makedirs(drive_path, exist_ok=True)

# Paths to ZIP files
male_zip = "/content/drive/MyDrive/CP/ta_in_male.zip"
female_zip = "/content/drive/MyDrive/CP/ta_in_female.zip"

def extract_and_rename(zip_path, prefix, num_files):
    """Extract ZIP, pick 'num_files' randomly, rename, and save to drive_path."""
    temp_extract = "temp_extracted"

    # Extract files
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(temp_extract)

    # Get all WAV files
    all_files = [f for f in os.listdir(temp_extract) if f.endswith(".wav")]

    # Randomly select required number of files
    selected_files = random.sample(all_files, num_files)

    # Rename and move files to Google Drive
    for idx, filename in enumerate(selected_files, start=1):
        new_name = f"{prefix}_{idx:03d}.wav"
        shutil.move(os.path.join(temp_extract, filename), os.path.join(drive_path, new_name))

    # Clean up temp folder
    shutil.rmtree(temp_extract)

# Process both male and female ZIPs
extract_and_rename(male_zip, "male", 20)
extract_and_rename(female_zip, "female", 20)

print(f"Renaming and selection complete! Files saved in '{drive_path}'")


Renaming and selection complete! Files saved in '/content/drive/MyDrive/New/Clean-vocals'


In [ ]:
!pip install musdb

In [ ]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Set path inside Google Drive
drive_path = "/content/drive/MyDrive/MUSDB18_Instrumentals"
os.makedirs(drive_path, exist_ok=True)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import musdb
import os
import soundfile as sf  # Import soundfile for writing audio

# Set path to store MUSDB18 dataset
musdb_path = "/content/musdb18_dataset"  # Local path for processing
os.makedirs(musdb_path, exist_ok=True)

# Set path inside Google Drive
drive_path = "/content/drive/MyDrive/MUSDB18_Instrumentals"
os.makedirs(drive_path, exist_ok=True)

# Download and extract the dataset
mus = musdb.DB(root=musdb_path, subsets="train", download=True)

# Extract instrumental (background music) and save to Google Drive
for i, track in enumerate(mus.tracks):
    instrumental = track.audio[:, 0]  # Convert stereo to mono
    file_path = os.path.join(drive_path, f"musdb_bg_{i}.wav")

    # Write the audio file
    sf.write(file_path, instrumental, track.rate)

    print(f"Saved: {file_path}")

print("Extracted MUSDB18 instrumentals successfully & saved to Google Drive!")


Saved: /content/drive/MyDrive/MUSDB18_Instrumentals/musdb_bg_0.wav
Saved: /content/drive/MyDrive/MUSDB18_Instrumentals/musdb_bg_1.wav
Saved: /content/drive/MyDrive/MUSDB18_Instrumentals/musdb_bg_2.wav
Saved: /content/drive/MyDrive/MUSDB18_Instrumentals/musdb_bg_3.wav
Saved: /content/drive/MyDrive/MUSDB18_Instrumentals/musdb_bg_4.wav
Saved: /content/drive/MyDrive/MUSDB18_Instrumentals/musdb_bg_5.wav
Saved: /content/drive/MyDrive/MUSDB18_Instrumentals/musdb_bg_6.wav
Saved: /content/drive/MyDrive/MUSDB18_Instrumentals/musdb_bg_7.wav
Saved: /content/drive/MyDrive/MUSDB18_Instrumentals/musdb_bg_8.wav
Saved: /content/drive/MyDrive/MUSDB18_Instrumentals/musdb_bg_9.wav
Saved: /content/drive/MyDrive/MUSDB18_Instrumentals/musdb_bg_10.wav
Saved: /content/drive/MyDrive/MUSDB18_Instrumentals/musdb_bg_11.wav
Saved: /content/drive/MyDrive/MUSDB18_Instrumentals/musdb_bg_12.wav
Saved: /content/drive/MyDrive/MUSDB18_Instrumentals/musdb_bg_13.wav
Saved: /content/drive/MyDrive/MUSDB18_Instrumentals/musdb_

In [ ]:
!pip install librosa numpy pydub soundfile


In [ ]:
import librosa
import numpy as np
import soundfile as sf
import os
from itertools import cycle

# Paths
drive_path = "/content/drive/MyDrive/CP/MUSDB18_Instrumentals"  # Background music path
vocals_path = "/content/drive/MyDrive/New/Clean-vocals"  # Path where clean vocals are stored
output_path = "/content/drive/MyDrive/Augmented - Audios"  # Save augmented audio
os.makedirs(output_path, exist_ok=True)

# List all vocal and instrumental files
vocal_files = sorted([f for f in os.listdir(vocals_path) if f.endswith(".wav")])
music_files = sorted([f for f in os.listdir(drive_path) if f.endswith(".wav")])

# Use cycle to loop through background music continuously
music_cycle = cycle(music_files)

def change_pitch(y, sr, n_steps):
    """Change the pitch of the audio."""
    return librosa.effects.pitch_shift(y, sr=sr, n_steps=n_steps)

def change_volume(y, gain_db):
    """Change the volume by a given gain in dB."""
    return y * (10**(gain_db / 20))

def mix_audio(vocals, music, vocal_vol=1.0, music_vol=0.5):
    """Mix vocals and music with specified volume adjustments."""
    vocals = change_volume(vocals, vocal_vol)
    music = change_volume(music, music_vol)
    return np.clip(vocals + music, -1.0, 1.0)  # Normalize to prevent clipping

def add_noise(audio, noise_type="white", noise_level=0.1):
    """Add different types of noise to the audio."""
    noise = np.random.randn(len(audio))  # White noise by default

    if noise_type == "pink":
        # Generate pink noise (1/f noise)
        uneven = len(noise) % 2
        fft = np.fft.rfft(noise)
        frequencies = np.fft.rfftfreq(len(noise))
        fft = fft / (frequencies + 1e-4)  # Reduce higher frequencies
        noise = np.fft.irfft(fft)
        if uneven:
            noise = noise[:-1]
    elif noise_type == "gaussian":
        noise = np.random.normal(0, 1, len(audio))  # Gaussian noise

    # Ensure noise length matches audio length
    if len(noise) < len(audio):
        noise = np.pad(noise, (0, len(audio) - len(noise)), mode='constant')  # Pad with zeros
    else:
        noise = noise[:len(audio)]  # Trim extra length

    noise = noise / (np.max(np.abs(noise)) + 1e-6)  # Normalize noise
    return np.clip(audio + noise * noise_level, -1.0, 1.0)  # Scale noise intensity

# Process vocals in order, cycling through noise types
noise_types = ["white", "pink", "gaussian"]
for i, vocal_file in enumerate(vocal_files):
    vocal_path = os.path.join(vocals_path, vocal_file)
    vocal, sr = librosa.load(vocal_path, sr=44100)

    # Get a background music track in a cyclic manner
    music_file = next(music_cycle)
    music_path = os.path.join(drive_path, music_file)
    music, _ = librosa.load(music_path, sr=44100)

    # Make sure both audio files are the same length
    min_length = min(len(vocal), len(music))
    vocal, music = vocal[:min_length], music[:min_length]

    # Assign one type of noise per vocal file (cycling through)
    noise_type = noise_types[i % len(noise_types)]

    # Generate variations
    for pitch_shift in [-2, 0, 2]:  # Lower, original, and higher pitch
        for vocal_vol in [-3, 0, 3]:  # Lower, normal, and boosted volume
            for music_vol in [-5, -3, 0]:  # Background music volume levels
                # Apply transformations
                augmented_vocal = change_pitch(vocal, sr, pitch_shift)
                augmented_vocal = add_noise(augmented_vocal, noise_type, noise_level=0.05)  # Lower noise level
                augmented_music = change_pitch(music, sr, pitch_shift)
                mixed_audio = mix_audio(augmented_vocal, augmented_music, vocal_vol, music_vol)

                # Save file
                new_filename = f"{vocal_file[:-4]}_{music_file[:-4]}_p{pitch_shift}_vv{vocal_vol}_mv{music_vol}_n{noise_type}.wav"
                sf.write(os.path.join(output_path, new_filename), mixed_audio, sr)
                print(f"Saved: {new_filename}")

print("Augmentation completed & saved to Google Drive!")


Saved: female_001_musdb_bg_0_p-2_vv-3_mv-5_nwhite.wav
Saved: female_001_musdb_bg_0_p-2_vv-3_mv-3_nwhite.wav
Saved: female_001_musdb_bg_0_p-2_vv-3_mv0_nwhite.wav
Saved: female_001_musdb_bg_0_p-2_vv0_mv-5_nwhite.wav
Saved: female_001_musdb_bg_0_p-2_vv0_mv-3_nwhite.wav
Saved: female_001_musdb_bg_0_p-2_vv0_mv0_nwhite.wav
Saved: female_001_musdb_bg_0_p-2_vv3_mv-5_nwhite.wav
Saved: female_001_musdb_bg_0_p-2_vv3_mv-3_nwhite.wav
Saved: female_001_musdb_bg_0_p-2_vv3_mv0_nwhite.wav
Saved: female_001_musdb_bg_0_p0_vv-3_mv-5_nwhite.wav
Saved: female_001_musdb_bg_0_p0_vv-3_mv-3_nwhite.wav
Saved: female_001_musdb_bg_0_p0_vv-3_mv0_nwhite.wav
Saved: female_001_musdb_bg_0_p0_vv0_mv-5_nwhite.wav
Saved: female_001_musdb_bg_0_p0_vv0_mv-3_nwhite.wav
Saved: female_001_musdb_bg_0_p0_vv0_mv0_nwhite.wav
Saved: female_001_musdb_bg_0_p0_vv3_mv-5_nwhite.wav
Saved: female_001_musdb_bg_0_p0_vv3_mv-3_nwhite.wav
Saved: female_001_musdb_bg_0_p0_vv3_mv0_nwhite.wav
Saved: female_001_musdb_bg_0_p2_vv-3_mv-5_nwhite.wav
Sa

In [ ]:
import os

def count_wav_files(folder, extension=".wav"):
    count = sum(1 for root, _, files in os.walk(folder) for file in files if file.endswith(extension))
    return count

folder_path = "/content/drive/MyDrive/Augmented - Audios"
num_wav_files = count_wav_files(folder_path)

print(f"Number of WAV files: {num_wav_files}")


Number of WAV files: 1080


In [ ]:
import os
import shutil

# Define original folders
clean_vocals_folder = "/content/drive/MyDrive/New/Clean-vocals"
background_music_folder = "/content/drive/MyDrive/CP/MUSDB18_Instrumentals"
mixed_audio_folder = "/content/drive/MyDrive/Augmented - Audios"

# Define new dataset structure
dataset_root = "/content/drive/MyDrive/New/Demucs_Training_Dataset"
train_path = os.path.join(dataset_root, "train")
valid_path = os.path.join(dataset_root, "valid")

# Create necessary directories
for subset in ["train", "valid"]:
    for category in ["mixture", "vocals", "background_music"]:
        os.makedirs(os.path.join(dataset_root, subset, category), exist_ok=True)

# Get sorted lists of files
clean_vocals = sorted(os.listdir(clean_vocals_folder))
background_music = sorted(os.listdir(background_music_folder))
mixed_audio = sorted(os.listdir(mixed_audio_folder))

# Split dataset (80% Train, 20% Validation)
split_index = int(len(mixed_audio) * 0.8)

# Function to move files
def move_files(files, src_folder, dest_folder, prefix):
    for i, file in enumerate(files):
        src_path = os.path.join(src_folder, file)
        dst_path = os.path.join(dest_folder, f"{prefix}_{i:03d}.wav")  # Rename to consistent format
        shutil.copy(src_path, dst_path)

# Move files into train & valid sets
move_files(mixed_audio[:split_index], mixed_audio_folder, os.path.join(train_path, "mixture"), "track")
move_files(clean_vocals[:split_index], clean_vocals_folder, os.path.join(train_path, "vocals"), "track")
move_files(background_music[:split_index], background_music_folder, os.path.join(train_path, "background_music"), "track")

move_files(mixed_audio[split_index:], mixed_audio_folder, os.path.join(valid_path, "mixture"), "track")
move_files(clean_vocals[split_index:], clean_vocals_folder, os.path.join(valid_path, "vocals"), "track")
move_files(background_music[split_index:], background_music_folder, os.path.join(valid_path, "background_music"), "track")

print("✅ Dataset structured for Demucs training!")


✅ Dataset structured for Demucs training!
